In [1]:
import sys
sys.path.insert(0,'../')

from src.model_training.datasets.experiments_sanitize.complete_sanitization import DefinitionDataset, DefinitionTestSet

In [2]:
train_set, val_set = DefinitionDataset.create_dataset(None, shuffle=False, seed=42)

Map:   0%|          | 0/288148 [00:00<?, ? examples/s]

Filter:   0%|          | 0/288022 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

13131


Filter:   0%|          | 0/288013 [00:00<?, ? examples/s]

Map:   0%|          | 0/35664 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35652 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

1505


Filter:   0%|          | 0/35652 [00:00<?, ? examples/s]

In [18]:
1530 / 35722 

0.042830748558311406

In [3]:
test_set = DefinitionTestSet.create_dataset(None, shuffle=False, seed=42)

Map:   0%|          | 0/35738 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35726 [00:00<?, ? examples/s]

0it [00:00, ?it/s]

1530


Filter:   0%|          | 0/35722 [00:00<?, ? examples/s]

In [14]:
import os

base_path = "/home/jfeil/MasterThesis/dataset_distillation/base-v5-unshuffled"
train_set.to_parquet(os.path.join(base_path, "train.parquet"))
test_set.to_parquet(os.path.join(base_path, "test.parquet"))
val_set.to_parquet(os.path.join(base_path, "val.parquet"))


Creating parquet from Arrow format:   0%|          | 0/275 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

14604652

In [4]:
from datasets import Dataset
import os

base_path = "/home/jfeil/MasterThesis/dataset_distillation/base-v5-unshuffled"
train_set = Dataset.from_parquet(os.path.join(base_path, "train.parquet"))
val_set = Dataset.from_parquet(os.path.join(base_path, "val.parquet"))
test_set = Dataset.from_parquet(os.path.join(base_path, "test.parquet"))

In [5]:
train_set[0]

{'title': 'Subregnum',
 'context_word': 'Subregnum',
 'context_sentence': 'Die Taxonomie der Biologie geht auf die Systematik Carl von Linnés zurück. Hier finden wir Begriffe wie Regnum - Subregnum - Divisio und Subdivisio, mit anderen Worten Reich - Unterreich - Abteilung und Unterabteilung',
 'gt': 'Biologie, Systematik: nächstkleinere Unterteilung der taxonomischen Regna (Reiche)',
 'prompt': 'Die Taxonomie der Biologie geht auf die Systematik Carl von Linnés zurück. Hier finden wir Begriffe wie Regnum - Subregnum - Divisio und Subdivisio, mit anderen Worten Reich - Unterreich - Abteilung und Unterabteilung Was ist die Definition von Subregnum? '}

In [6]:
from src.prompting import default_experiments

In [7]:
from tqdm.auto import tqdm
import os
import asyncio
from openai import AsyncOpenAI, OpenAI

base_url="http://localhost:8080/v1"

async_client = AsyncOpenAI(
    base_url=base_url,
    api_key="-"
)
client = OpenAI(
    base_url=base_url,
    api_key="-"
)


def create_messages(data, experiment):
    return [
            {
                "role": "system",
                "content": " ".join(experiment.system_prompt)
            },
            *experiment.create_examples(),
            {
                "role": "user",
                "content": experiment.question_prompt % (data['context_sentence'], data['context_word']),
            }
        ]


async def async_prompt(row, experiment) -> None:
    return (await async_client.chat.completions.create(
        messages=create_messages(row, experiment),
        model="tgi",
        stream=False,
        max_tokens=512, 
        frequency_penalty=1, 
        logprobs=False, 
        seed=42, 
        temperature=0.5,

    )).choices[0].message.content

def sync_prompt(row, experiment):
    return client.chat.completions.create(
            model="tgi",
            messages=create_messages(row, experiment),
        ).choices[0].message.content


async def prompt_dataset(dataset, experiment, batch_size=512, warm_up=32):
    responses = []
    tasks = []
    for i in tqdm(dataset):
        tasks.append(async_prompt(i, experiment))
    responses.append(await asyncio.gather(*tasks[0:warm_up]))
    
    for i in tqdm(range(warm_up, len(dataset), batch_size)):
        responses.append(await asyncio.gather(*tasks[i:i + batch_size]))
    return responses

# asyncio.run(main())

In [6]:
data = {
    'context_word': "Pferdewurst",
    'context_sentence': "\"Dort liegt zwischen den Steinproben eine dicke rote Wurst - Pferdewurst, ein Geschenk Watzeks, der sie morgens für mich hinterlassen hat.\"",
}

await async_prompt(data, default_experiments[0])

'Art von Wurst, die aus Pferdemuskel hergestellt wird'

In [10]:
await async_prompt(train_set[0], default_experiments[0])

'höchste Unterstufe der Taxonomie, über Abteilung und unter Reich'

In [12]:

# resp = []
# for i in test_set.select(range(100)):
#     resp += (sync_prompt(i, default_experiments[0]), )
# print(resp)

# 20.66s

async def create_output(dataset, experiment, split):
    resp = await prompt_dataset(dataset, experiment)
    resp = [x for xs in resp for x in xs]
    
    import json
    
    with open(f'../ModelDestillationExperiments_{split}_Llama3-1-80B.json', 'w+') as outfile:
        json.dump([(f"{b['prompt']}", a) for a, b in zip(resp, dataset)], outfile)

In [13]:
await create_output(test_set, default_experiments[0], "test")
await create_output(val_set, default_experiments[0], "val")
await create_output(train_set, default_experiments[0], "train")

  0%|          | 0/34189 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/34146 [00:00<?, ?it/s]

  0%|          | 0/67 [00:00<?, ?it/s]

CancelledError: 

In [9]:
default_experiments[0].question_prompt

'%s: Was ist in diesem Kontext die Definition von %s?'

In [6]:
# test_set
# # batch_size 64 -> 25min
# # batch_size 512 -> 21min 30s
# # batch_size 512 + 32 warm_up -> 11min 49s
# # batch_size 512 + 32 warm_up -> 9min 1s

train_set_responses = await prompt_dataset(train_set, default_experiments[0])
train_set_responses = [x for xs in train_set_responses for x in xs]

import json

with open('../ModelDestillationExperiments_TRAIN_Llama3-1-80B.json', 'w+') as outfile:
    json.dump([(f"{b['prompt']}", a) for a, b in zip(train_set_responses, train_set)], outfile)

  0%|          | 0/99207 [00:00<?, ?it/s]

  0%|          | 0/194 [00:00<?, ?it/s]

In [7]:
import json

with open('../ModelDestillationExperiments_TRAIN_Llama3-1-80B.json', 'w+') as outfile:
    json.dump([(f"{b['prompt']}", a) for a, b in zip(train_set_responses, train_set)], outfile)

In [8]:
val_set_responses = await prompt_dataset(val_set, default_experiments[0])
val_set_responses = [x for xs in val_set_responses for x in xs]

import json

with open('../ModelDestillationExperiments_VAL_Llama3-1-80B.json', 'w+') as outfile:
    json.dump([(f"{b['prompt']}", a) for a, b in zip(val_set_responses, val_set)], outfile)

  0%|          | 0/12360 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [11]:
len(responses)

195

In [7]:
responses = [x for xs in responses for x in xs]

In [8]:
import pandasgui



pandasgui.show([(f"{b['prompt']}", a) for a, b in zip(responses, test_set)])

PandasGUI INFO — pandasgui.gui — Opening PandasGUI
INFO:pandasgui.gui:Opening PandasGUI


In [9]:
import json

with open('../ModelDestillationExperiments_Llama3-1-80B.json', 'w+') as outfile:
    json.dump([(f"{b['prompt']}", a) for a, b in zip(responses, test_set)], outfile)

In [11]:
default_experiments[1].create_examples()

[{'role': 'user',
  'content': '"Die Liebe überwindet alle Grenzen": Was ist in diesem Kontext die Definition von Liebe? '},
 {'role': 'assistant',
  'content': 'inniges Gefühl der Zuneigung für jemanden oder für etwas'},
 {'role': 'user',
  'content': '"Natürlich sind diese Stifte stabil und robust genug, um den täglichen Rettungseinsatz absolvieren zu können.": Was ist in diesem Kontext die Definition von Stifte? '},
 {'role': 'assistant', 'content': 'Rekrut'}]

In [11]:
import json

with open("/home/jfeil/MasterThesis/ModelDestillationExperiments_train_Llama3-1-80B.json") as file:
    data = json.load(file)
    
with open("/home/jfeil/MasterThesis/ModelDestillationExperiments_train_PRETTY_Llama3-1-80B.json", "w+") as file:
    json.dump(data, file, indent=4)

In [31]:
# train_set, val_set, test_set

base_path = "../dataset_distillation/v1"

def write_split(data_set, split, base_path=base_path, overwrite=False):
    if not os.path.exists(base_path):
        os.mkdir(base_path)
    path = os.path.join(base_path, f"{split}.parquet")
    if os.path.exists(path):
        if overwrite:
            os.remove(path)
        else:
            raise Exception("Already existing!")
    
    with open(f"/home/jfeil/MasterThesis/ModelDestillationExperiments_{split}_Llama3-1-80B.json") as file:
        data = json.load(file)
    
    data_set = data_set.rename_column("gt", "wiktionary_gt")
    data_set = data_set.add_column("gt", [d[1] for d in data])
    data_set.to_parquet(path)

write_split(train_set, "train")
write_split(test_set, "test")
write_split(val_set, "val")


Creating parquet from Arrow format:   0%|          | 0/275 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

In [28]:
write_split(val_set, "val")

Creating parquet from Arrow format:   0%|          | 0/35 [00:00<?, ?ba/s]

In [29]:
write_split(test_set, "test")

Flattening the indices:   0%|          | 0/34189 [00:00<?, ? examples/s]

ValueError: Failed to concatenate on axis=1 because tables don't have the same number of rows

In [36]:
from datasets import Dataset

test = Dataset.from_parquet("/home/jfeil/MasterThesis/dataset_distillation/v0/train.parquet")

Generating train split: 0 examples [00:00, ? examples/s]